# [Projet 3: Développez un moteur de recommandations de films](https://openclassrooms.com/projects/developpez-un-moteur-de-recommandations-de-films)

Link to raw data: [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Moteur+de+recommandation+de+films/imdb-5000-movie-dataset.zip).

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'/Documents/GitHub/OCDataSciencePath/Project3/')    # @home
elif HOST == 'Sirius.local':
    os.chdir(HOME+'Perso/GitHub/OCDataSciencePath/Project3/')         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import pandas as pd

In [2]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToData = HOME+'xxx'    # @home
elif HOST == 'Sirius.local':
    pathToData = HOME+'Downloads/movie_metadata.csv'         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
df = pd.read_csv(pathToData)

In [3]:
n,p = df.shape
print(n,p)
df.sample(5)

5043 28


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
2500,Color,John Fortenberry,63.0,82.0,12.0,281.0,Chris Kattan,8000.0,30324946.0,Comedy|Music|Romance,...,239.0,English,USA,PG-13,1.700000e+07,1998.0,357.0,6.2,1.85,0
1641,Color,Bobby Farrelly,162.0,92.0,101.0,760.0,Larry David,971.0,44328238.0,Comedy,...,189.0,English,USA,PG,NaN,2012.0,860.0,5.1,1.85,13000
4767,Color,Patrick Gilles,NaN,90.0,0.0,569.0,Dylan Baker,970.0,NaN,History,...,NaN,English,USA,NaN,NaN,2015.0,812.0,7.5,NaN,337
3419,Color,NaN,9.0,142.0,NaN,427.0,Jack O'Connell,27000.0,NaN,Drama|Romance,...,33.0,English,UK,NaN,NaN,NaN,698.0,7.7,NaN,0
3423,Color,Katsuhiro Ôtomo,150.0,124.0,78.0,4.0,Takeshi Kusao,6.0,439162.0,Action|Animation|Sci-Fi,...,430.0,Japanese,Japan,R,1.100000e+09,1988.0,5.0,8.1,1.85,0


In [24]:
for col in df.columns.sort_values():
    miss = 100 * (1 - df[col].count()/n)
    print('{:>30}: {}%'.format(col, miss))

        actor_1_facebook_likes: 0.0013880626611144642
                  actor_1_name: 0.0013880626611144642
        actor_2_facebook_likes: 0.0025778306563553066
                  actor_2_name: 0.0025778306563553066
        actor_3_facebook_likes: 0.004560777315090192
                  actor_3_name: 0.004560777315090192
                  aspect_ratio: 0.0652389450723776
                        budget: 0.09756097560975607
     cast_total_facebook_likes: 0.0
                         color: 0.00376759865159626
                content_rating: 0.06008328375966687
                       country: 0.0009914733293674427
       director_facebook_likes: 0.020622645250842786
                 director_name: 0.020622645250842786
                      duration: 0.002974419988102328
          facenumber_in_poster: 0.0025778306563553066
                        genres: 0.0
                         gross: 0.1752924846321634
                    imdb_score: 0.0
                      language: 0.00237953599

We only keep apriori interesting variables:

In [5]:
keepVar = ['movie_title',
           'plot_keywords','genres',
           'country','language','title_year',
           'duration','imdb_score',
           'actor_1_name','actor_2_name','actor_3_name','director_name']
df = df[keepVar]

In [7]:
df.sample(5)

,movie_title,plot_keywords,genres,country,language,duration,imdb_score,actor_1_name,actor_2_name,actor_3_name,director_name
4730,The Station Agent,abandoned|depot|dwarf|newfoundland new jersey|...,Comedy|Drama,USA,English,89.0,7.7,Peter Dinklage,Joe Lo Truglio,Paula Garcés,Tom McCarthy
2419,Spy Hard,1990s|dentist|hit with a door|martial arts|museum,Action|Comedy,USA,English,81.0,5.3,Hulk Hogan,Barry Bostwick,Nicollette Sheridan,Rick Friedberg
4501,Down and Out with the Dolls,female bassist|female drummer|female guitarist...,Comedy|Music,USA,English,88.0,6.1,Lemmy,Coyote Shivers,Zoë Poledouris,Kurt Voss
4448,Wind Walkers,after dark horrorfest,Action|Horror|Thriller,USA,English,93.0,3.6,Rudy Youngblood,Glen Powell,Kiowa Gordon,Russell Friedenberg
4779,The Toxic Avenger,boy|health club|superhero|toxic waste|troma,Action|Comedy|Horror|Sci-Fi,USA,English,91.0,6.2,Patrick Kilpatrick,Mark Torgl,Pat Ryan,Michael Herz


Needed post-processing:
- doublons/approximative doublons in `movie_title`,`genres`,`country`,`language`,`actor_N_name`,`director_name`
    - specific care for `movie_title`
- one-hot encode `plot_keywords` (see if not too large)
- one-hot encode `genres` (+ beware of approximative doublons)
- try one-hot encode the three `actor_N_name` (ignore order)
- check absurde values in `duration` and `imdb_score`

In [15]:
print(df['movie_title'].duplicated().sum())
df.loc[df['movie_title'].duplicated(),'movie_title']

126


137                          The Legend of Tarzan 
187     The Twilight Saga: Breaking Dawn - Part 2 
204                           Godzilla Resurgence 
303                                           Pan 
389                                Fantastic Four 
395                      The Fast and the Furious 
590                                      Hercules 
656                                  Total Recall 
794                                  The Avengers 
1122                                        Dredd 
1220                                     Unbroken 
1305                          Victor Frankenstein 
1332                     The Fast and the Furious 
1449                                    From Hell 
1508                                   The Lovers 
1582                                 Ghostbusters 
1805                              The Jungle Book 
1994                            Snakes on a Plane 
2020                                  Point Break 
2049                           

In [16]:
print(df.duplicated().sum())
df.loc[df.duplicated()]

121


,movie_title,plot_keywords,genres,country,language,duration,imdb_score,actor_1_name,actor_2_name,actor_3_name,director_name
137,The Legend of Tarzan,africa|capture|jungle|male objectification|tarzan,Action|Adventure|Drama|Romance,USA,English,110.0,6.6,Christoph Waltz,Alexander Skarsgård,Casper Crump,David Yates
187,The Twilight Saga: Breaking Dawn - Part 2,battle|friend|super strength|vampire|vision,Adventure|Drama|Fantasy|Romance,USA,English,115.0,5.5,Robert Pattinson,Kristen Stewart,Taylor Lautner,Bill Condon
204,Godzilla Resurgence,blood|godzilla|monster|sequel,Action|Adventure|Drama|Horror|Sci-Fi,Japan,Japanese,120.0,8.2,Mark Chinnery,Shin'ya Tsukamoto,Atsuko Maeda,Hideaki Anno
303,Pan,1940s|child hero|fantasy world|orphan|referenc...,Adventure|Family|Fantasy,USA,English,111.0,5.8,Hugh Jackman,Cara Delevingne,Nonso Anozie,Joe Wright
389,Fantastic Four,box office flop|critically bashed|portal|telep...,Action|Adventure|Sci-Fi,USA,English,100.0,4.3,Tim Blake Nelson,Reg E. Cathey,Tim Heidecker,Josh Trank
395,The Fast and the Furious,eighteen wheeler|illegal street racing|truck|t...,Action|Crime|Thriller,USA,English,106.0,6.7,Paul Walker,Vin Diesel,Jordana Brewster,Rob Cohen
590,Hercules,army|greek mythology|hercules|king|mercenary,Action|Adventure,USA,English,101.0,6.0,Dwayne Johnson,Rufus Sewell,Ingrid Bolsø Berdal,Brett Ratner
656,Total Recall,ambiguous ending|false memory|implanted memory...,Action|Sci-Fi,USA,English,113.0,7.5,Ronny Cox,Rachel Ticotin,Marshall Bell,Paul Verhoeven
794,The Avengers,alien invasion|assassin|battle|iron man|soldier,Action|Adventure|Sci-Fi,USA,English,173.0,8.1,Chris Hemsworth,Robert Downey Jr.,Scarlett Johansson,Joss Whedon
1122,Dredd,brutality|dark humor|female antagonist|female ...,Action|Sci-Fi,UK,English,95.0,7.1,Wood Harris,Jason Cope,Rakie Ayola,Pete Travis
